In [ ]:
import os
import cv2
import numpy as np
import torch
from torchvision import transforms
from torchvision.models.segmentation import deeplabv3_resnet101
from torchvision.transforms import Compose, ToTensor, Normalize
from PIL import Image
import matplotlib.pyplot as plt
from skimage import measure

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# [STEP 0] 이미지 로드
def load_image(image_path):
    img_orig = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)
    return img_orig, img_rgb

# [STEP 1] 가장 큰 객체 마스크 생성 (사람 또는 동물)
def get_largest_object_mask(image_bgr, class_id=15):
    model = deeplabv3_resnet101(pretrained=True).to(device).eval()
    transform = Compose([
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = transform(cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)['out'][0]
        mask = output.argmax(0).byte().cpu().numpy()

    object_mask = (mask == class_id).astype(np.uint8)

    labels = measure.label(object_mask)
    regions = measure.regionprops(labels)

    if not regions:
        return np.zeros_like(object_mask)

    largest_region = max(regions, key=lambda r: r.area)
    largest_object_mask = np.zeros_like(object_mask)
    largest_object_mask[labels == largest_region.label] = 1

    kernel = np.ones((5, 5), np.uint8)
    largest_object_mask = cv2.morphologyEx(largest_object_mask.astype(np.uint8), cv2.MORPH_CLOSE, kernel) * 255

    return largest_object_mask

# [STEP 2] 깊이 맵 생성
def estimate_depth_midas(img):
    midas = torch.hub.load("intel-isl/MiDaS", "DPT_Hybrid").to(device).eval()
    transform = torch.hub.load("intel-isl/MiDaS", "transforms").dpt_transform
    img_transformed = transform(img).to(device)

    with torch.no_grad():
        prediction = midas(img_transformed)
        depth = prediction.squeeze().cpu().numpy()

        depth_filtered = cv2.bilateralFilter(depth.astype(np.float32), d=9, sigmaColor=75, sigmaSpace=75)
        depth_normalized = cv2.normalize(depth_filtered, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

        return depth_normalized

# [STEP 3] 블러 및 Feather 적용
def apply_blur_with_feathering(img, core_mask, feather_mask, bg_mask):
    img_blur = cv2.GaussianBlur(img, (61, 61), 0)

    feather_alpha = feather_mask.astype(np.float32) / 255.0
    feather_alpha = feather_alpha[:, :, np.newaxis] * 0.8 + 0.2

    feather_blended = (img * feather_alpha + img_blur * (1 - feather_alpha)).astype(np.uint8)

    final_img = np.zeros_like(img)
    core_3ch = cv2.cvtColor(core_mask, cv2.COLOR_GRAY2BGR)
    
    final_img[core_3ch > 0] = img[core_3ch > 0]
    final_img[feather_alpha > 0] = feather_blended[feather_alpha > 0]

    return final_img

# [STEP 4] 기본 배경 교체
def replace_background(img, new_bg_path, core_mask, feather_mask):
    new_bg = cv2.imread(new_bg_path)
    new_bg_resized = cv2.resize(new_bg, (img.shape[1], img.shape[0]))

    feather_alpha = feather_mask.astype(np.float32) / 255.0
    feather_alpha = feather_alpha[:, :, np.newaxis]

    blended_bg = (img * feather_alpha + new_bg_resized * (1 - feather_alpha)).astype(np.uint8)

    final_img = np.zeros_like(img)
    core_3ch = cv2.cvtColor(core_mask, cv2.COLOR_GRAY2BGR)

    final_img[core_3ch > 0] = img[core_3ch > 0]
    final_img[core_3ch == 0] = blended_bg[core_3ch == 0]

    return final_img

# 실행 및 시각화 예제
if __name__ == "__main__":
    # 이미지 경로 설정
    image_path = os.path.join(os.getenv('USERPROFILE'), 'Desktop', 'image.png')
    
    # 배경 이미지 경로 설정
    new_background_path = os.path.join(os.getenv('USERPROFILE'), 'Desktop', 'background.jpg')
    
    # 이미지 로드 및 처리 단계 실행
    img_orig, img_rgb = load_image(image_path)
    
    mask_person_or_animal = get_largest_object_mask(img_orig) # 사람(class_id=15) 또는 동물(class_id 변경 가능)
    
    depth_map_generated = estimate_depth_midas(img_rgb)
    
    # Core/Feather/Background 분리 마스크 생성
    mask_blur_processed = cv2.GaussianBlur(mask_person_or_animal, (31, 31), 0)
    
    core_mask_final   = (mask_blur_processed >150


C:\Users\hjin0\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\hjin0\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using cache found in C:\Users\hjin0/.cache\torch\hub\intel-isl_MiDaS_master
C:\Users\hjin0\AppData\Local\Packages\PythonSoftwareFoundation.Python.3

**변환 전 결과물1**
-함수적용

-![a1.png](./a1.png)

![a2.png](./a2.png)

-적용 후 이미지
![a3.png](./a3.png)

**변환 전 결과물2**
-함수적용

-![a01.png](./a01.png)

![a02.png](./a02.png)

-적용 후 이미지
![a03.png](./a03.png)

**변환 후 결과물1**
-함수적용

-![b1.png](./b1.png)

![b2.png](./b2.png)

-적용 후 이미지
![b3.png](./b3.png)

**변환 후 결과물2**
-함수적용

-![b01.png](./b01.png)

![b02.png](./b02.png)

-적용 후 이미지
![b03.png](./b03.png)